In [284]:
#!pip install sklearn

In [147]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [148]:
df_insta_likes = pd.read_csv('../data/df_insta_timeseries.csv')
df_transfer = pd.read_csv('../data/player_dynamic.csv')
df_noten = pd.read_csv('../data/player_noten.csv')
df_dream = pd.read_csv('../data/player_scores_parsed.csv')

In [149]:
def normalize_name(name):
    if name == 'David von Ballmoos':
        return 'David Von Ballmoos'
    elif name == 'Cedric Zesiger':
        return 'Cédric Zesiger'
    elif name == 'Mohamed Camara':
        return 'Mohamed Ali Camara'
    elif name == 'Jean Pierre Nsame':
        return 'Jean-Pierre Nsame'   
    elif name == 'Christopher Martins':
        return 'Christopher Martins Pereira'   
    else:
        return name

In [150]:
df_dream['name'] = df_dream['player_name'].apply(lambda x: normalize_name(x))
df_noten['name'] = df_noten['name'].apply(lambda x: normalize_name(x))
df_transfer['name'] = df_transfer['name'].apply(lambda x: normalize_name(x))
df_insta_likes['name'] = df_insta_likes['player'].apply(lambda x: normalize_name(x))
df_dream = df_dream.drop('player_name', axis=1)
df_insta_likes = df_insta_likes.drop('player', axis=1)

In [151]:
df_dream['date'] = pd.to_datetime(df_dream['date']) 
df_noten['date'] = pd.to_datetime(df_noten['date']) 
df_transfer['date'] = pd.to_datetime(df_transfer['date']) 
df_insta_likes['date'] = pd.to_datetime(df_insta_likes['post_date']) 
df_insta_likes = df_insta_likes.drop('post_date', axis=1)
df_noten['note'] = df_noten['note'].astype(float)
#  format='%d%b%Y:%H:%M:%S.%f'

In [179]:
def resample_player(df_player):
    #print(df_player['date'].min())
    #display(df_player.sort_values('date'))
    #display(df_player)
    df_player_ = df_player.drop_duplicates(subset=['date'], keep='first')
    df_player = df_player_.set_index('date')
    resampled = df_player.resample('W').fillna('nearest')
    resampled = resampled.drop('name', axis=1)
    return resampled.reset_index()

In [180]:
df_dream_res = df_dream.groupby('name').apply(lambda x:resample_player(x))

df_noten_res = df_noten.groupby('name').apply(lambda x:resample_player(x))
df_transfer_res = df_transfer.groupby('name').apply(lambda x:resample_player(x))
df_insta_likes_res = df_insta_likes.groupby('name').apply(lambda x:resample_player(x))

df_dream_res = df_dream_res.reset_index()[['name','date','score']]
df_noten_res = df_noten_res.reset_index()[['name','date','note']]
df_transfer_res = df_transfer_res.reset_index()[['name','date','market_value']]
df_insta_likes_res = df_insta_likes_res.reset_index()[['name','date','likes']]

In [183]:
df_fame = pd.merge(df_noten_res, df_insta_likes_res,  how='outer', left_on=['date','name'], right_on=['date','name'])
df_performance = pd.merge(df_dream_res, df_transfer_res,  how='outer', left_on=['date','name'], right_on=['date','name'])

df_fame = df_fame.fillna(method='ffill')

df_performance = df_performance.fillna(method='ffill')
df_performance = df_performance.fillna(method='bfill')

## Normalize columns

In [184]:
def min_max_normalize(column):
    return (column-column.min())/(column.max()-column.min())

In [185]:
df_fame['note_norm'] = min_max_normalize(df_fame['note'])
df_fame['likes_norm'] = min_max_normalize(df_fame['likes'])

df_performance['market_value_norm'] = min_max_normalize(df_performance['market_value'])
df_performance['score_norm'] = min_max_normalize(df_performance['score'])

In [186]:
df_fame['fame'] = df_fame['note_norm']*1.5 + df_fame['likes_norm']
df_performance['performance'] = df_performance['market_value_norm'] + df_performance['score_norm'] 

## Save to CSV

In [187]:
df_performance.to_csv( '../data/df_dynamic_performance_merged.csv', index=False)
df_fame.to_csv( '../data/df_dynamic_fame_merged.csv', index=False)